BDLE 2023 TP2

révision: 19/09/2023

# TP2 2023 Fenetre Foursquare avec le système DuckDB (ETUDIANT)


# Préparation

In [ ]:
import pandas as pd
from google.colab import data_table

def display(df, n=100):
  return data_table.DataTable(df.limit(n).df(), include_index=False, num_rows_per_page=10)

In [ ]:
!pip install duckdb==0.8.1

# Démarrer le service DuckDB

In [ ]:
import duckdb

# db = duckdb.connect('foursquare')
db = duckdb.connect(':memory:')

# vérifier que le service fonctrionne
r = db.sql("SELECT 'hello' as col")
display(r)

,col
0,hello


# Données

In [ ]:
import os
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

['dataset_TSMC2014_NYC.zip', 'dataset_TSMC2014_NYC.csv']

URL pour l'accès aux datasets

In [ ]:
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4"
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


In [ ]:
import os
from urllib import request
import zipfile

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

def unzip_file(local_dir, file):
  with zipfile.ZipFile(local_dir + "/" + file, 'r') as zip_ref:
    zip_ref.extractall(local_dir)
  # os.remove(local_dir + "/" + file)


web_dir = PUBLIC_DATASET + "/foursquare"

# ce fichier vient de kaggle : https://www.kaggle.com/datasets/chetanism/foursquare-nyc-and-tokyo-checkin-dataset
zip_filename = "dataset_TSMC2014_NYC.zip"


download_file(web_dir, local_dir, zip_filename)
unzip_file(local_dir, zip_filename)

# Liste des fichiers
os.listdir(local_dir)

dataset_TSMC2014_NYC.zip is already stored


['dataset_TSMC2014_NYC.zip', 'dataset_TSMC2014_NYC.csv']

In [ ]:
!head -4 /local/data/dataset_TSMC2014_NYC.csv

userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.71981038,-74.00258103,-240,Tue Apr 03 18:00:09 +0000 2012
979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.60679958,-74.04416981,-240,Tue Apr 03 18:00:25 +0000 2012
69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.71616168,-73.88307006,-240,Tue Apr 03 18:02:24 +0000 2012


# Table Visit

In [ ]:
# duckdb.read_csv('/local/data/notes1M.json')

stmt = """
DROP TABLE if exists Visit;
CREATE TABLE Visit as
  SELECT *
  FROM '/local/data/dataset_TSMC2014_NYC.csv';
"""
db.sql(stmt)
print("done")

done


## Afficher le contenu d'une table

In [ ]:
visit = db.table('Visit')
visit

┌────────┬──────────────────────┬──────────────────────┬───┬──────────────┬────────────────┬──────────────────────┐
│ userId │       venueId        │   venueCategoryId    │ … │  longitude   │ timezoneOffset │     utcTimestamp     │
│ int64  │       varchar        │       varchar        │   │    double    │     int64      │       varchar        │
├────────┼──────────────────────┼──────────────────────┼───┼──────────────┼────────────────┼──────────────────────┤
│    470 │ 49bbd6c0f964a520f4…  │ 4bf58dd8d48988d127…  │ … │ -74.00258103 │           -240 │ Tue Apr 03 18:00:0…  │
│    979 │ 4a43c0aef964a520c6…  │ 4bf58dd8d48988d1df…  │ … │ -74.04416981 │           -240 │ Tue Apr 03 18:00:2…  │
│     69 │ 4c5cc7b485a1e21e00…  │ 4bf58dd8d48988d103…  │ … │ -73.88307006 │           -240 │ Tue Apr 03 18:02:2…  │
│    395 │ 4bc7086715a7ef3bef…  │ 4bf58dd8d48988d104…  │ … │ -73.98251878 │           -240 │ Tue Apr 03 18:02:4…  │
│     87 │ 4cf2c5321d18a14395…  │ 4bf58dd8d48988d1cb…  │ … │ -73.9896583

In [ ]:
display(visit,3)

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012


## Schéma d'une table

In [ ]:
db.sql("""

SELECT column_name, data_type
FROM duckdb_columns
where table_name = 'Visit'

""")

┌─────────────────┬───────────┐
│   column_name   │ data_type │
│     varchar     │  varchar  │
├─────────────────┼───────────┤
│ userId          │ BIGINT    │
│ venueId         │ VARCHAR   │
│ venueCategoryId │ VARCHAR   │
│ venueCategory   │ VARCHAR   │
│ latitude        │ DOUBLE    │
│ longitude       │ DOUBLE    │
│ timezoneOffset  │ BIGINT    │
│ utcTimestamp    │ VARCHAR   │
└─────────────────┴───────────┘

# Requêtes

convertir le timestamp

In [ ]:
visit2 = db.sql("""
  SELECT *, strptime(utcTimestamp, '%a %b %d %X %z %Y') as datetime
  FROM Visit
""")
visit2

┌────────┬──────────────────────┬───┬────────────────┬──────────────────────┬──────────────────────┐
│ userId │       venueId        │ … │ timezoneOffset │     utcTimestamp     │       datetime       │
│ int64  │       varchar        │   │     int64      │       varchar        │ timestamp with tim…  │
├────────┼──────────────────────┼───┼────────────────┼──────────────────────┼──────────────────────┤
│    470 │ 49bbd6c0f964a520f4…  │ … │           -240 │ Tue Apr 03 18:00:0…  │ 2012-04-03 18:00:0…  │
│    979 │ 4a43c0aef964a520c6…  │ … │           -240 │ Tue Apr 03 18:00:2…  │ 2012-04-03 18:00:2…  │
│     69 │ 4c5cc7b485a1e21e00…  │ … │           -240 │ Tue Apr 03 18:02:2…  │ 2012-04-03 18:02:2…  │
│    395 │ 4bc7086715a7ef3bef…  │ … │           -240 │ Tue Apr 03 18:02:4…  │ 2012-04-03 18:02:4…  │
│     87 │ 4cf2c5321d18a14395…  │ … │           -240 │ Tue Apr 03 18:03:0…  │ 2012-04-03 18:03:0…  │
│    484 │ 4b5b981bf964a52090…  │ … │           -240 │ Tue Apr 03 18:04:0…  │ 2012-04-03 18

In [ ]:
visit3 = db.sql("""
  SELECT userId, venueId, datetime
  FROM visit2
""")

visit3

┌────────┬──────────────────────────┬──────────────────────────┐
│ userId │         venueId          │         datetime         │
│ int64  │         varchar          │ timestamp with time zone │
├────────┼──────────────────────────┼──────────────────────────┤
│    470 │ 49bbd6c0f964a520f4531fe3 │ 2012-04-03 18:00:09+00   │
│    979 │ 4a43c0aef964a520c6a61fe3 │ 2012-04-03 18:00:25+00   │
│     69 │ 4c5cc7b485a1e21e00d35711 │ 2012-04-03 18:02:24+00   │
│    395 │ 4bc7086715a7ef3bef9878da │ 2012-04-03 18:02:41+00   │
│     87 │ 4cf2c5321d18a143951b5cec │ 2012-04-03 18:03:00+00   │
│    484 │ 4b5b981bf964a520900929e3 │ 2012-04-03 18:04:00+00   │
│    642 │ 4ab966c3f964a5203c7f20e3 │ 2012-04-03 18:04:38+00   │
│    292 │ 4d0cc47f903d37041864bf55 │ 2012-04-03 18:04:42+00   │
│    428 │ 4ce1863bc4f6a35d8bd2db6c │ 2012-04-03 18:06:18+00   │
│    877 │ 4be319b321d5a59352311811 │ 2012-04-03 18:06:19+00   │
│     ·  │            ·             │           ·              │
│     ·  │            ·  

##  Ex1 Sequences de visites



### Numéro de POI

Définir les vues
 * venueId_poi(venueId, poi) qui associe le numéro original de venueId avec poi.
   * Indication, utiliser la fonction row_number() over( ...)
 * visit_poi(userId, poi, datetime) avec des numéros de poi allant de 1 à n



In [ ]:
venueIds = db.sql("""
  Select distinct venueId
  From Visit


""")


venueId_poi = db.sql("""
  select venueId, row_number() over (order by venueId) as poi

  from venueIds

""")
venueId_poi

┌──────────────────────────┬───────┐
│         venueId          │  poi  │
│         varchar          │ int64 │
├──────────────────────────┼───────┤
│ 3fd66200f964a52000e71ee3 │     1 │
│ 3fd66200f964a52000e81ee3 │     2 │
│ 3fd66200f964a52000f11ee3 │     3 │
│ 3fd66200f964a52001e51ee3 │     4 │
│ 3fd66200f964a52001e81ee3 │     5 │
│ 3fd66200f964a52002eb1ee3 │     6 │
│ 3fd66200f964a52003e51ee3 │     7 │
│ 3fd66200f964a52003e71ee3 │     8 │
│ 3fd66200f964a52003e81ee3 │     9 │
│ 3fd66200f964a52004e41ee3 │    10 │
│            ·             │     · │
│            ·             │     · │
│            ·             │     · │
│ 4b4f4d1bf964a5205a0027e3 │  9991 │
│ 4b4f593ef964a5207f0227e3 │  9992 │
│ 4b4f59def964a520a20227e3 │  9993 │
│ 4b4f5a31f964a520b20227e3 │  9994 │
│ 4b4f5dd8f964a520600327e3 │  9995 │
│ 4b4f601df964a520cf0327e3 │  9996 │
│ 4b4f6600f964a520cf0427e3 │  9997 │
│ 4b4f676cf964a520150527e3 │  9998 │
│ 4b4f676ff964a520160527e3 │  9999 │
│ 4b4f695ef964a520640527e3 │ 10000 │
├

In [ ]:

visit_poi = db.sql("""
  select v.userId, vi.poi, v.datetime
  from visit2 v, venueId_poi vi
  where v.venueId=vi.venueId

""")
visit_poi

┌────────┬───────┬──────────────────────────┐
│ userId │  poi  │         datetime         │
│ int64  │ int64 │ timestamp with time zone │
├────────┼───────┼──────────────────────────┤
│    470 │  2389 │ 2012-04-03 18:00:09+00   │
│    979 │  3922 │ 2012-04-03 18:00:25+00   │
│     69 │ 20329 │ 2012-04-03 18:02:24+00   │
│    395 │ 15115 │ 2012-04-03 18:02:41+00   │
│     87 │ 23551 │ 2012-04-03 18:03:00+00   │
│    484 │ 10590 │ 2012-04-03 18:04:00+00   │
│    642 │  6366 │ 2012-04-03 18:04:38+00   │
│    292 │ 23900 │ 2012-04-03 18:04:42+00   │
│    428 │ 23304 │ 2012-04-03 18:06:18+00   │
│    877 │ 16382 │ 2012-04-03 18:06:19+00   │
│     ·  │    ·  │           ·              │
│     ·  │    ·  │           ·              │
│     ·  │    ·  │           ·              │
│    744 │  6789 │ 2012-07-08 14:10:18+00   │
│    293 │ 24571 │ 2012-07-08 14:12:31+00   │
│     11 │  3318 │ 2012-07-08 14:16:53+00   │
│    908 │  7276 │ 2012-07-08 14:25:08+00   │
│    348 │  6789 │ 2012-07-08 14:2

### 1.1) Rang
Pour chaque utilisateur, ordonner les visites par date et leur attribuer un **rang** allant de 1 (plus ancienne) à n (la plus récente)

In [ ]:
visit_rank = db.sql("""
  select userId, poi, datetime, row_number() over(partition by userId order by datetime) as rank
  from visit_poi
  order by userId, datetime

""")
visit_rank

┌────────┬───────┬──────────────────────────┬───────┐
│ userId │  poi  │         datetime         │ rank  │
│ int64  │ int64 │ timestamp with time zone │ int64 │
├────────┼───────┼──────────────────────────┼───────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │
│      1 │   525 │ 2012-04-10 00:24:31+00   │     6 │
│      1 │  1241 │ 2012-04-10 03:36:56+00   │     7 │
│      1 │ 33237 │ 2012-04-10 16:21:48+00   │     8 │
│      1 │  2643 │ 2012-04-12 17:19:21+00   │     9 │
│      1 │ 33237 │ 2012-04-13 15:41:41+00   │    10 │
│      · │    ·  │           ·              │     · │
│      · │    ·  │           ·              │     · │
│      · │    ·  │           ·              │     · │
│     56 │  5511 │ 2012-12-22 01:22:30+00   │   308 │
│     56 │  3681 │ 2012-12-2

### 1.2) Date précédente et déplacement
 On veut connaitre la durée de déplacement séparant deux visites consécutives d'un meme utilisateur.
Pour chaque visite d'un utilisateur, ajouter l'attribut **prev_date** contenant la date de la visite qu'il a effectuée précédemment.

Puis ajouter l'attribut **interval_duration** calculé par différence entre la date courante et la date précédente.
Indication, voir la fonction *date_diff('sec', a, b)*

In [ ]:
visit_prev_date = db.sql("""
  select userId, poi, datetime, rank, first(datetime) over (partition by userId order by datetime rows between 1 preceding and current row) prev_date
  from visit_rank
  order by userId, datetime

""")
visit_prev_date

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 2012-04-08 20:02:10+00   │
│      1 │   525 │ 2012-04-10 00:24:31+00   │     6 │ 2012-04-09 16:20:52+00   │
│      1 │  1241 │ 2012-04-10 03:36:56+00   │     7 │ 2012-04-10 00:24:31+00   │
│      1 │ 33237 │ 2012-04-10 16:21:48+00   │     8 │ 2012-04-10 03:36:56+00   │
│      1 │  2643 │ 2012-04-1

In [ ]:
visit_duration = db.sql("""
  select userId, poi, datetime, rank, prev_date, date_diff('sec', prev_date, datetime) as interval_duration
  from visit_prev_date

""")

visit_duration

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┬───────────────────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │ interval_duration │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │       int64       │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┼───────────────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │                 0 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │            238253 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │             88685 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │              6101 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 2012-04-08 20:02:10+00   │             73122 │
│      1 │   525 │ 2012-04-10 00:24:31+00   │     6 │ 2012-04-09 16:20:52+00   │           

### 1.3) Numéro de trajectoire

Séparer les visites de chaque utilisateur en trajectoires.

  * Dans une trajectoire, la durée entre deux visites consécutives ne peut pas dépasser 1 jour (soit 24h * 3600 secondes).

Ajouter l'attribut **traj** indiquant le numéro de la trajectoire pour un utilisateur donné.
Indication: on peut utiliser une expression *case when then else end*

In [ ]:
visit_traj1 = db.sql("""
  select userId, poi, datetime, rank, prev_date, interval_duration, case when interval_duration>=24*3600 or interval_duration=0 then 1 else 0 end as debut
  from visit_duration
""")

visit_traj1

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┬───────────────────┬───────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │ interval_duration │ debut │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │       int64       │ int32 │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┼───────────────────┼───────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │                 0 │     1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │            238253 │     1 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │             88685 │     1 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │              6101 │     0 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 2012-04-08 20:02:10+00   │             73122 │     0 │
│      1 │   525 │ 

In [ ]:
visit_traj2 = db.sql("""
  select userId, poi, datetime, rank, prev_date, interval_duration, debut, sum(debut) over (partition by userId order by userId, datetime) as traj
  from visit_traj1
  order by userId, datetime

""")
visit_traj2

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┬───────────────────┬───────┬────────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │ interval_duration │ debut │  traj  │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │       int64       │ int32 │ int128 │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┼───────────────────┼───────┼────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │                 0 │     1 │      1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │            238253 │     1 │      2 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │             88685 │     1 │      3 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │              6101 │     0 │      3 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 20

## Ex2: Fenètres glissantes

### 2.1) Nombre de POI visités en une semaine

Ajouter l'attribut **nb_POI_7jours** donnant pour chaque jour où un utilisateur a fait des visites, le nombre de POI visités dans la semaine qui précède.

In [ ]:
visit_semaine = db.sql("""
  select userId, poi, datetime,  count(poi) over (
                                                partition by userId
                                                order by datetime
                                                range between interval 7 day preceding and current row)
                                                as nb_POI_7jours
  from visit_poi
  order by userId, datetime
""")
visit_semaine

┌────────┬───────┬──────────────────────────┬───────────────┐
│ userId │  poi  │         datetime         │ nb_POI_7jours │
│ int64  │ int64 │ timestamp with time zone │     int64     │
├────────┼───────┼──────────────────────────┼───────────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │             1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │             2 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │             3 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │             4 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │             5 │
│      1 │   525 │ 2012-04-10 00:24:31+00   │             6 │
│      1 │  1241 │ 2012-04-10 03:36:56+00   │             7 │
│      1 │ 33237 │ 2012-04-10 16:21:48+00   │             8 │
│      1 │  2643 │ 2012-04-12 17:19:21+00   │             8 │
│      1 │ 33237 │ 2012-04-13 15:41:41+00   │             9 │
│      · │    ·  │           ·              │             · │
│      · │    ·  │           ·              │             · │
│      ·

### 2.2) Nombre cumulé de visites par utilisateur

Pour chaque utilisateur ajouter un attribut **cumul_visites** indiquant le nombre de visites qu'il a effectuées

In [ ]:
visit_cumul = db.sql("""
  select userId, poi, count(poi) over (partition by userId order by datetime rows between unbounded preceding and current row) as cumul_visites
  from visit_poi
  order by userId, datetime
""")

visit_cumul

┌────────┬───────┬───────────────┐
│ userId │  poi  │ cumul_visites │
│ int64  │ int64 │     int64     │
├────────┼───────┼───────────────┤
│      1 │  6428 │             1 │
│      1 │ 24554 │             2 │
│      1 │ 26196 │             3 │
│      1 │  4131 │             4 │
│      1 │   963 │             5 │
│      1 │   525 │             6 │
│      1 │  1241 │             7 │
│      1 │ 33237 │             8 │
│      1 │  2643 │             9 │
│      1 │ 33237 │            10 │
│      · │    ·  │             · │
│      · │    ·  │             · │
│      · │    ·  │             · │
│     56 │  5511 │           308 │
│     56 │  3681 │           309 │
│     56 │ 31764 │           310 │
│     57 │ 15816 │             1 │
│     57 │  5651 │             2 │
│     57 │ 15816 │             3 │
│     57 │ 21139 │             4 │
│     57 │ 25176 │             5 │
│     57 │  5651 │             6 │
│     57 │ 15816 │             7 │
├────────┴───────┴───────────────┤
│ ? rows (>9999 rows

## Exercice 3

### 3.1) Visites dans une trajectoire
On constate qu'une trajectoire peut contenir plusieurs visites **consécutives** d'un même POI et on veut "fusionner" ces visites.
Pour chaque trajectoire, agréger les visites consécutives d'un meme POI. Définir les attributs date_début, date_fin pour la visite d'un POI.

Rmq1: s'il n'y a pas plusieurs visites consécutives pour un POI alors ses dates de début et de fin sont identiques.

Rmq2: une trajectoire peut contenir plusieurs visites non consécutives d'un même POI.

In [ ]:
premiere_visite = db.sql("""
  select userId, poi, datetime, case
                                  when lag(poi) over (partition by userId order by datetime) = poi then 0
                                  else 1
                                end as premiere
  from visit_poi
""")

premiere_visite

┌────────┬───────┬──────────────────────────┬──────────┐
│ userId │  poi  │         datetime         │ premiere │
│ int64  │ int64 │ timestamp with time zone │  int32   │
├────────┼───────┼──────────────────────────┼──────────┤
│     11 │ 17338 │ 2012-04-23 11:39:18+00   │        1 │
│     11 │  4480 │ 2012-04-23 23:39:27+00   │        1 │
│     11 │ 17621 │ 2012-04-23 23:42:28+00   │        1 │
│     11 │ 23697 │ 2012-04-24 13:09:12+00   │        1 │
│     11 │ 21454 │ 2012-04-25 12:22:49+00   │        1 │
│     11 │  7432 │ 2012-05-22 13:39:14+00   │        1 │
│     11 │  1892 │ 2012-05-25 16:43:47+00   │        1 │
│     11 │ 25815 │ 2012-05-27 22:30:06+00   │        1 │
│     11 │  5056 │ 2012-05-28 01:40:10+00   │        1 │
│     11 │  1293 │ 2012-05-28 13:00:49+00   │        1 │
│      · │    ·  │           ·              │        · │
│      · │    ·  │           ·              │        · │
│      · │    ·  │           ·              │        · │
│    479 │ 35809 │ 2012-10-23 1

In [ ]:
rank_visit = db.sql("""
  select userId, poi, datetime, premiere, sum(premiere) over (partition by userId order by datetime rows between unbounded preceding and current row) visit_rank
  from premiere_visite
""")

rank_visit

┌────────┬───────┬──────────────────────────┬──────────┬────────────┐
│ userId │  poi  │         datetime         │ premiere │ visit_rank │
│ int64  │ int64 │ timestamp with time zone │  int32   │   int128   │
├────────┼───────┼──────────────────────────┼──────────┼────────────┤
│     11 │ 17338 │ 2012-04-23 11:39:18+00   │        1 │          1 │
│     11 │  4480 │ 2012-04-23 23:39:27+00   │        1 │          2 │
│     11 │ 17621 │ 2012-04-23 23:42:28+00   │        1 │          3 │
│     11 │ 23697 │ 2012-04-24 13:09:12+00   │        1 │          4 │
│     11 │ 21454 │ 2012-04-25 12:22:49+00   │        1 │          5 │
│     11 │  7432 │ 2012-05-22 13:39:14+00   │        1 │          6 │
│     11 │  1892 │ 2012-05-25 16:43:47+00   │        1 │          7 │
│     11 │ 25815 │ 2012-05-27 22:30:06+00   │        1 │          8 │
│     11 │  5056 │ 2012-05-28 01:40:10+00   │        1 │          9 │
│     11 │  1293 │ 2012-05-28 13:00:49+00   │        1 │         10 │
│      · │    ·  │  

In [ ]:
visites_aggregees = db.sql("""
  select userId, poi, visit_rank, first(datetime) as date_debut, last(datetime) as date_fin
  from rank_visit
  group by userId, poi, visit_rank
""")

visites_aggregees

┌────────┬───────┬────────────┬──────────────────────────┬──────────────────────────┐
│ userId │  poi  │ visit_rank │        date_debut        │         date_fin         │
│ int64  │ int64 │   int128   │ timestamp with time zone │ timestamp with time zone │
├────────┼───────┼────────────┼──────────────────────────┼──────────────────────────┤
│    313 │   238 │          7 │ 2012-04-08 02:23:31+00   │ 2012-04-08 02:23:31+00   │
│    313 │  1420 │          8 │ 2012-04-08 05:54:25+00   │ 2012-04-08 05:54:25+00   │
│    313 │  7095 │         14 │ 2012-04-11 01:32:54+00   │ 2012-04-11 01:32:54+00   │
│    313 │ 16849 │         26 │ 2012-04-14 22:41:42+00   │ 2012-04-14 22:41:42+00   │
│    313 │ 17807 │         30 │ 2012-04-18 00:02:30+00   │ 2012-04-18 00:02:30+00   │
│    313 │ 12952 │         32 │ 2012-04-18 20:30:58+00   │ 2012-04-18 20:30:58+00   │
│    313 │ 23469 │         34 │ 2012-04-21 01:31:39+00   │ 2012-04-21 01:31:39+00   │
│    313 │ 28002 │         38 │ 2012-04-23 19:44:52+00

afficher les visites d'une trajectoire pour laqelle au moins une fusion a été faite.

In [ ]:
verif1 = db.sql("""
  select *
  from visites_aggregees a
  where date_debut!=date_fin
  limit 1
""")
verif1

┌────────┬───────┬────────────┬──────────────────────────┬──────────────────────────┐
│ userId │  poi  │ visit_rank │        date_debut        │         date_fin         │
│ int64  │ int64 │   int128   │ timestamp with time zone │ timestamp with time zone │
├────────┼───────┼────────────┼──────────────────────────┼──────────────────────────┤
│     15 │ 28212 │        165 │ 2012-09-13 13:34:10+00   │ 2012-09-14 12:53:21+00   │
└────────┴───────┴────────────┴──────────────────────────┴──────────────────────────┘

In [ ]:
verif = db.sql("""
  select *
  from rank_visit
  where userId=15 and poi=28212
""")
verif

┌────────┬───────┬──────────────────────────┬──────────┬────────────┐
│ userId │  poi  │         datetime         │ premiere │ visit_rank │
│ int64  │ int64 │ timestamp with time zone │  int32   │   int128   │
├────────┼───────┼──────────────────────────┼──────────┼────────────┤
│     15 │ 28212 │ 2012-09-05 12:33:27+00   │        1 │        158 │
│     15 │ 28212 │ 2012-09-13 13:34:10+00   │        1 │        165 │
│     15 │ 28212 │ 2012-09-14 12:53:21+00   │        0 │        165 │
└────────┴───────┴──────────────────────────┴──────────┴────────────┘